In [1]:
import numpy as np
import pandas as pd


In [2]:
# Load Excel file (Install openpyxl for loading (!pip install openpyxl))
df = pd.read_excel("D:\\IndiaAI\\Champions Code\\Dataset\\Champion_Data.xlsx")
#df[['Gims_CategoryName','GrievanceDescription', 'FinalReply']].head(15)
df.head(15)

,GrievanceNo,Gims_CategoryName,State,CreatedDate,ReplyStatus,GrievanceDescription,FinalReply,FinalReplyDate
0,UP00135828,UAM/Udyam Registration/Certificate related issues,UTTAR PRADESH,2024-12-30,Replied,Please find the attachment of Letter Head,"Sir,&#13;&#10; Please upload the clear im...",2024-12-31
1,BR00135827,UAM/Udyam Registration/Certificate related issues,BIHAR,2024-12-30,Replied,Udyam no Udyam-br-31-0038425 My address i...,"Sir,&#13;&#10;As per your request you are advi...",2024-12-31
2,MH00135826,UAM/Udyam Registration/Certificate related issues,MAHARASHTRA,2024-12-30,Replied,"To, The Udyog Aadhaar Registration Authority,...",&#2350;&#2361;&#2379;&#2342;&#2351;/&#2350;&#2...,2025-01-01
3,CG00135825,UAM/Udyam Registration/Certificate related issues,CHHATTISGARH,2024-12-30,Replied,"This is regarding Udyam Registration, even aft...",With the provided PAN the registered Udyam is ...,2025-01-03
4,CG00135824,UAM/Udyam Registration/Certificate related issues,CHHATTISGARH,2024-12-30,Replied,The grievance is regarding my Udyam Registrati...,With the provided PAN the registered Udyam is ...,2025-01-03
5,JH00135823,UAM/Udyam Registration/Certificate related issues,JHARKHAND,2024-12-30,Replied,"DEAR SIR, I AM A PROPRIETOR HAVING PAN: GZDPD7...","Madam,&#13;&#10;This is regarding your grievan...",2025-01-02
6,OD00135822,UAM/Udyam Registration/Certificate related issues,ODISHA,2024-12-30,Replied,When I apply for udyam registration.. It shows...,"Sir, the following information is available in...",2024-12-31
7,UP00135821,UAM/Udyam Registration/Certificate related issues,UTTAR PRADESH,2024-12-30,Replied,सर / मैडम मैं सुभम अग्रवाल पुत्र फते चन्द्र ...,"Sir,&#13;&#10; As per our records an UAM is re...",2024-12-31
8,RJ00135820,UAM/Udyam Registration/Certificate related issues,RAJASTHAN,2024-12-30,Replied,I am running a Cement Business and want to get...,Your Udyam registration Number is UDYAM-RJ-17...,2025-01-01
9,RJ00135819,Related to DCMSME Scheme,RAJASTHAN,2024-12-30,Replied,Sir mera pm vishwakarma yojna ka OTP nhi aa rh...,The Tool kit will be distributed through India...,2025-01-01


In [3]:
# Print categories in Gims_CategoryName column and their counts
df['Gims_CategoryName'].value_counts()

Gims_CategoryName
UAM/Udyam Registration/Certificate related issues                    7663
Delayed Payment/MSEFC related issues                                  524
Loans from Banks                                                      194
Related to Scheme of KVIC                                             105
Related to MSME Scheme                                                 89
Related to  MSME-DFO                                                   72
MSME Act 2006 related Matters                                          48
Any other banking issue                                                42
Related to Public Procurement by PSUs                                  28
Related to DCMSME Scheme                                               28
Related to GST                                                         25
Related to various State Government                                    24
Insurance Claim related issues                                         16
Definition of MSMEs 

In [4]:
!pip -q install deep-translator langdetect

In [7]:
# Translating GrievanceDescription & FinalReply to English (Hindi/Hinglish/other → English) including lowercasing
# Uses existing libraries: deep-translator (GoogleTranslator) + langdetect for language detection.
# Note: Translation requires internet access and may be rate-limited by the provider.
# Install langauge detection and translation libraries if not already installed:(!pip install deep-translator langdetect)
import re
from functools import lru_cache
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException

CATEGORY_COL = "Gims_CategoryName"
DESC_COL = "GrievanceDescription"
REPLY_COL = "FinalReply"

# Quick script detection for Hindi (Devanagari block)
DEVANAGARI_RE = re.compile(r"[\u0900-\u097F]")

def _safe_str(x) -> str:
    if x is None or (isinstance(x, float) and pd.isna(x)) or pd.isna(x):
        return ""
    return str(x)

@lru_cache(maxsize=50000)
def _translate_cached(text: str) -> str:
    # GoogleTranslator auto-detects source when source='auto'
    return GoogleTranslator(source="auto", target="en").translate(text)

def to_english(text: str) -> str:
    text = _safe_str(text).strip()
    if not text:
        return ""
    # If the text contains Hindi characters, translate.
    if DEVANAGARI_RE.search(text):
        try:
            return _translate_cached(text)
        except Exception:
            return text
    # Otherwise, detect language; translate if not English.
    try:
        lang = detect(text)
    except LangDetectException:
        lang = "unknown"
    if lang and lang != "en" and lang != "unknown":
        try:
            return _translate_cached(text)
        except Exception:
            return text
    # Hinglish may be detected as 'en' sometimes; you can force-translate later if needed.
    return text

# Defensive checks
missing_cols = [c for c in [DESC_COL, REPLY_COL] if c not in df.columns]
if missing_cols:
    raise KeyError(f"Missing expected columns in df: {missing_cols}. Available columns: {list(df.columns)}")

# Translate into English columns
df[CATEGORY_COL] = df[CATEGORY_COL].apply(to_english)
df[DESC_COL] = df[DESC_COL].apply(to_english)
df[REPLY_COL] = df[REPLY_COL].apply(to_english)

# Optional: standardize to lowercase after translation (English output)
df[CATEGORY_COL] = df[CATEGORY_COL].str.lower()
df[DESC_COL] = df[DESC_COL].str.lower()
df[REPLY_COL] = df[REPLY_COL].str.lower()
df[[CATEGORY_COL,DESC_COL, REPLY_COL]].head(10)

,Gims_CategoryName,GrievanceDescription,FinalReply
0,uam/udyam registration/certificate related issues,please find the attachment of letter head,"sir,&#13;&#10; please upload the clear im..."
1,uam/udyam registration/certificate related issues,udyam no udyam-br-31-0038425 my address i...,"sir,&#13;&#10;as per your request you are advi..."
2,uam/udyam registration/certificate related issues,"to, the udyog aadhaar registration authority,...",&#2350;&#2361;&#2379;&#2342;&#2351;/&#2350;&#2...
3,uam/udyam registration/certificate related issues,"this is regarding udyam registration, even aft...",with the provided pan the registered udyam is ...
4,uam/udyam registration/certificate related issues,the grievance is regarding my udyam registrati...,with the provided pan the registered udyam is ...
5,uam/udyam registration/certificate related issues,"dear sir, i am a proprietor having pan: gzdpd7...","madam,&#13;&#10;this is regarding your grievan..."
6,uam/udyam registration/certificate related issues,when i apply for udyam registration.. it shows...,"sir, the following information is available in..."
7,uam/udyam registration/certificate related issues,sir/madam i am subham aggarwal son of fateh ch...,"sir,&#13;&#10; as per our records an uam is re..."
8,uam/udyam registration/certificate related issues,i am running a cement business and want to get...,your udyam registration number is udyam-rj-17...
9,related to dcmsme scheme,"sir, i have not received the otp of pm vishwak...",the tool kit will be distributed through india...


In [ ]:
!pip -q install nltk pyspellchecker contractions

In [8]:
# Advanced preprocessing for clustering (English-focused)
# Applies to GrievanceDescription & FinalReply:
# 1) remove stopwords
# 2) spelling correction
# 3) chat/slang treatment via contractions expansion (library)
# 4) URL removal
# 5) remove punctuation
# 6) remove numbers
# 7) handle repeated characters (e.g., 'pleeeease' -> 'pleease')

import string
import nltk
from nltk.corpus import stopwords
import contractions
from spellchecker import SpellChecker

# Ensure NLTK stopwords are available
try:
    _ = stopwords.words("english")
except LookupError:
    nltk.download("stopwords")

EN_STOPWORDS = set(stopwords.words("english"))

# Whitelist: domain terms/acronyms we do NOT want spell-corrected
DOMAIN_KEEP = {
    "msme", "udyam", "urn", "nic", "gst", "gstin", "pan", "aadhaar", "aadhar", "upi",
    "llp", "otp", "ifsc", "gov", "india", "portal",
}

URL_RE = re.compile(r"(https?://\S+|www\.\S+)", re.IGNORECASE)
NUM_RE = re.compile(r"\b\d+\b")
REPEATED_RE = re.compile(r"(.)\1{2,}")  # 3+ repeats -> reduce

PUNCT_TABLE = str.maketrans({ch: " " for ch in string.punctuation})

spell = SpellChecker(distance=1)

def _safe_str(x) -> str:
    if x is None or (isinstance(x, float) and pd.isna(x)) or pd.isna(x):
        return ""
    return str(x)

def _only_englishish_token(tok: str) -> bool:
    # Restrict spell correction to simple alphabetic tokens
    return tok.isalpha() and tok.isascii()

@lru_cache(maxsize=200000)
def _correct_token(tok: str) -> str:
    # Keep domain terms as-is
    if tok in DOMAIN_KEEP:
        return tok
    # Spell-correct only for reasonably sized words
    if len(tok) <= 3:
        return tok
    if not _only_englishish_token(tok):
        return tok
    corrected = spell.correction(tok)
    return corrected if corrected else tok

def preprocess_text(text: str) -> str:
    text = _safe_str(text).strip()
    if not text:
        return ""

    # Lowercase (English output expected after translation cell)
    text = text.lower()

    # 4) URL removal
    text = URL_RE.sub(" ", text)

    # 3) slang/chat treatment using existing library (contractions expansion)
    # Examples: "can't" -> "cannot", "i'm" -> "i am"
    # (This does not expand SMS shortcuts like u->you; add a small map only if needed later.)
    try:
        text = contractions.fix(text)
    except Exception:
        pass

    # 7) handle repeated characters (e.g., 'pleeeease' -> 'pleease')
    # Reduce 3+ repeats to 2 repeats (keeps some emphasis but normalizes)
    text = REPEATED_RE.sub(r"\1\1", text)

    # 6) remove standalone numbers
    text = NUM_RE.sub(" ", text)

    # 5) remove punctuation
    text = text.translate(PUNCT_TABLE)

    # Tokenize (simple whitespace)
    tokens = [t for t in text.split() if t]

    # 2) spelling correction (token-wise, cached)
    corrected = [_correct_token(t) for t in tokens]

    # 1) stopword removal
    filtered = [t for t in corrected if t not in EN_STOPWORDS]

    return " ".join(filtered).strip()

# Defensive checks
missing_cols = [c for c in [DESC_COL, REPLY_COL] if c not in df.columns]
if missing_cols:
    raise KeyError(f"Missing expected columns in df: {missing_cols}. Available columns: {list(df.columns)}")
df[CATEGORY_COL] = df[CATEGORY_COL].apply(preprocess_text)
df[DESC_COL] = df[DESC_COL].apply(preprocess_text)
df[REPLY_COL] = df[REPLY_COL].apply(preprocess_text)

# Preview
df[[CATEGORY_COL,DESC_COL, REPLY_COL]].head(10)

,Gims_CategoryName,GrievanceDescription,FinalReply
0,uam udyam registration certificate related issues,please find attachment letter head,sir please upload clear image aadhar card pan ...
1,uam udyam registration certificate related issues,udyam udyam br address wrong want cancel prere...,sir per request advised send details signed co...
2,uam udyam registration certificate related issues,udyog aadhaar registration authority dear sir ...,grievance change registered mobile number requ...
3,uam udyam registration certificate related issues,regarding udyam registration even multiple att...,provided pan registered udyam udyam cg satan k...
4,uam udyam registration certificate related issues,grievance regarding udyam registration even mu...,provided pan registered udyam udyam cg satan k...
5,uam udyam registration certificate related issues,dear sir proprietor pan gzdpd7433l aadhar numb...,madam regarding grievance registered champions...
6,uam udyam registration certificate related issues,apply udyam registration shows udyam registrat...,sir following information available database d...
7,uam udyam registration certificate related issues,sir madam subham aggarwal son fate chandra for...,sir per records uam registered given pan detai...
8,uam udyam registration certificate related issues,running cement business want get registered ms...,udyam registration number udyam rj verify udya...
9,related dcmsme scheme,sir received otp pm vishwakarma yojana take to...,tool kit distributed india post per guideline ...


In [20]:
# Filter the main category (largest number of entries)
dfx = df[df[CATEGORY_COL] == 'uam udyam registration certificate related issues']

# New dataframe with first 100 rows of dfx
dfx_100 = dfx.head(100).copy().reset_index(drop=True)

dfx_100.shape

# Sub-Categorization of GrievanceDescription which are related to 'UAM Udyam Registration Certificate Related Issues'

(100, 8)

In [2]:
from transformers import pipeline

# load zero-shot-classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
# category definitions
candidate_labels = [
    "Email Id, Mobile Number Update related issue",
    "aadhar/PAN Update related issues",
    "PAN already used in the registration"
    "Cancel request for Udyam Certificate Registration",
    "Udyam Registration done Yet not getting Registration ID",
    "Update Udyam registration id",
    "Issues in certificate Downloading",
    "Fake MSME Certificate related Issues",
    "latitude / longitude map Issues",
    "change of address",
    "OTP validation Authentication Error",
    "other issues"
]

dfx = pd.DataFrame(dfx).reset_index(drop=True)

results = []
for desc in dfx_100[DESC_COL].tolist():
    output = classifier(desc, candidate_labels)
    results.append(output)
    
# show results
for res in results[:5]:
    print(res)

: 

In [10]:
import re

# Define subcategory rules as (subcategory, [keywords/phrases])
SUBCATEGORY_RULES = [
    ("Email Id, Mobile Number Update related issue", [r"email", r"mail", r"mobile", r"phone", r"number", r"update"]),
    ("aadhar/PAN Update related issues", [r"aadhar", r"aadhaar", r"pan", r"update", r"correction", r"change"]),
    ("Cancel request for Udyam Certificate Registration", [r"cancel", r"withdraw", r"delete", r"remove"]),
    ("change of address", [r"address", r"change", r"update", r"edit"]),
    ("latitude / longitude map Issues", [r"latitude", r"longitude", r"map", r"location", r"geo"]),
    ("Issues in certificate Download", [r"certificate", r"download", r"print", r"unable", r"issue"]),
    ("Update Udyam registration id", [r"udyam", r"registration id", r"update", r"change"]),
    ("PAN already used in the registration", [r"pan", r"already used", r"duplicate", r"existing"]),
    ("Udyam Registration done Yet not getting Registration ID", [r"registration done", r"not getting", r"id", r"missing", r"not received"]),
    ("Fake MSME Certificate related Issues", [r"fake", r"fraud", r"invalid", r"msme", r"certificate"]),
    ("OTP validation Authentication Error", [r"otp", r"validation", r"authentication", r"error", r"not received", r"failed"])
]

def assign_subcategory(text):
    text = str(text).lower()
    for subcat, patterns in SUBCATEGORY_RULES:
        for pat in patterns:
            if re.search(pat, text):
                return subcat
    return "Other/Unclassified"

# Apply subcategory assignment to dfx
dfx["GrievanceSubcategory"] = dfx["GrievanceDescription"].apply(assign_subcategory)

# Preview the result
dfx[["GrievanceDescription", "GrievanceSubcategory"]].head(20)

C:\Users\hp\AppData\Local\Temp\ipykernel_16580\1791938285.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx["GrievanceSubcategory"] = dfx["GrievanceDescription"].apply(assign_subcategory)


,GrievanceDescription,GrievanceSubcategory
0,please find attachment letter head,Other/Unclassified
1,udyam udyam br address wrong want cancel prere...,Cancel request for Udyam Certificate Registration
2,udyog aadhaar registration authority dear sir ...,"Email Id, Mobile Number Update related issue"
3,regarding udyam registration even multiple att...,aadhar/PAN Update related issues
4,grievance regarding udyam registration even mu...,aadhar/PAN Update related issues
5,dear sir proprietor pan gzdpd7433l aadhar numb...,"Email Id, Mobile Number Update related issue"
6,apply udyam registration shows udyam registrat...,aadhar/PAN Update related issues
7,sir madam subham aggarwal son fate chandra for...,"Email Id, Mobile Number Update related issue"
8,running cement business want get registered ms...,"Email Id, Mobile Number Update related issue"
10,dear sir madam great day uan number tn16a00914...,"Email Id, Mobile Number Update related issue"


In [34]:
# Auto-tag grievances into Category + Subcategory (rule-based) 
from typing import Tuple, Optional

def _norm(x: Optional[object]) -> str:
    if x is None:
        return ""
    if isinstance(x, float) and pd.isna(x):
        return ""
    s = str(x)
    s = s.lower()
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Each rule: (Category, Subcategory, [regex patterns])
# Order matters: first match wins.
RULES = [
    # 1) Eligibility and Documentation Queries
    (
        "Eligibility and Documentation Queries",
        "Eligibility Criteria",
        [
            r"\beligib(le|ility)\b",
            r"\b(investment|turnover)\b",
            r"\b(micro|small|medium)\b",
            r"\bmsme\b",
            r"\bclassification\b",
        ],
    ),
    (
        "Eligibility and Documentation Queries",
        "Mandatory Documents",
        [
            r"\b(document|documents)\b",
            r"\bupload\b",
            r"\bpaperless\b",
            r"\bself\s*-?declaration\b",
            r"\baadhaar\b",
            r"\bpan\b",
            r"\bbank\s*(details|account)\b",
        ],
    ),
    (
        "Eligibility and Documentation Queries",             #-----------------------------
        "Aadhaar/PAN Update",
        [
            r"\b(document|documents)\b",
            r"\bupload\b",
            r"\baadhar update?aadhaar update?\b",
            r"\bpan update?\b",
            r"\baadhaar registration?pan registration?registration\b",
            r"\baadhaar\b",
            r"\bpan\b",
            r"\bbank\s*(details|account)\b",
        ],
    ),
    (
        "Eligibility and Documentation Queries",             #-----------------------------
        "Mobile No./email Update",
        [
            r"\bemail update?\b",
            r"\bmobile update? mobile number?number\b",
            r"\bmobile\b",
            r"\bphone?phone number?number\b",
            r"\bemail\b",
        ],
    ),
    (
        "Eligibility and Documentation Queries",
        "GSTIN Requirement",
        [r"\bgst(in)?\b", r"\bgstin\b"],
    ),
    (
        "Eligibility and Documentation Queries",
        "Business Type",
        [
            r"\bpropriet(or|orship)\b",
            r"\bpartnership\b",
            r"\bllp\b",
            r"\b(trader|retail|wholesale)\b",
            r"\bprivate\s+limited\b",
            r"\bcompany\b",
        ],
    ),

    # 2) Technical and Portal Issues
    (
        "Technical and Portal Issues",
        "OTP Issues",
        [
            r"\botp\b",
            r"one\s*time\s*password",
            r"not\s*receiv(ing|ed)\s*otp",
            r"did\s*not\s*receiv(e|ed)\s*otp",
            r"\baadhaar\s*linked\b",
        ],
    ),
    (
        "Technical and Portal Issues",
        "PAN/Aadhaar Mismatch",
        [
            r"\bmismatch\b",
            r"pan\s*.*aadhaar|aadhaar\s*.*pan",
            r"\bname\s*mismatch\b",
            r"\bdob\s*mismatch\b",
        ],
    ),
    (
        "Technical and Portal Issues",
        "Portal Errors",
        [
            r"\b404\b",
            r"\b(error|errors)\b",
            r"\bserver\b",
            r"\bdown(time)?\b",
            r"not\s*loading",
            r"unable\s*to\s*(open|access)",
            r"something\s*went\s*wrong",
        ],
    ),
    (
        "Technical and Portal Issues",
        "Certificate Download",
        [
            r"\bdownload\b",
            r"\bcertificate\b",
            r"\budyam\b.*\bcertificate\b|\bcertificate\b.*\budyam\b",
            r"\bprint\b",
            r"\bemailed\b|\bemail\b",
        ],
    ),
    (
        "Technical and Portal Issues",
        "URN Not Found",
        [
            r"\burn\b",
            r"\budyam\s*registration\s*number\b",
            r"\bnot\s*found\b",
            r"\binvalid\b",
            r"\bretrieve\b",
        ],
    ),
    (
        "Technical and Portal Issues",
        "NIC Code Selection",
        [
            r"\bnic\s*code\b",
            r"national\s+industrial\s+classification",
            r"\bbusiness\s*activity\b",
            r"\bactivity\s*code\b",
        ],
    ),
    (
        "Technical and Portal Issues",
        "Mobile Number Change / Aadhaar Link",
        [
            r"\bchange\s*mobile\b|\bmobile\s*number\s*change\b",
            r"\bupdate\s*mobile\b",
            r"\bmobile\s*number\b.*\b(aadhaar|link)\b",
        ],
    ),
    (
        "Technical and Portal Issues",
        "PAN Correction / Update",
        [
            r"\bpan\b.*\b(correct|correction|update|change)\b",
            r"\b(correct|correction|update|change)\b.*\bpan\b",
        ],
    ),

    # 3) Post-Registration Benefits and Updates
    (
        "Post-Registration Benefits and Updates",
        "Benefits",
        [
            r"\bbenefit(s)?\b",
            r"\bsubsid(y|ies)\b",
            r"\bloan(s)?\b",
            r"\bcredit\b",
            r"\btax\b.*\b(exemption|rebate|benefit)\b",
            r"\bscheme(s)?\b",
        ],
    ),
    (
        "Post-Registration Benefits and Updates",
        "Validity and Renewal",
        [r"\brenew(al)?\b", r"\bvalid(ity)?\b", r"\blife\s*time\b"],
    ),
    (
        "Post-Registration Benefits and Updates",
        "Updating Details",
        [
            r"\bupdate\b|\bedit\b|\bmodify\b|\bchange\b",
            r"\baddress\b",
            r"\bbank\b",
            r"\bturnover\b",
            r"\badd\s+activity\b|\bnew\s+activity\b",
            r"\bemail\b",
            r"\bname\b",
        ],
    ),
    (
        "Post-Registration Benefits and Updates",
        "Multiple Registrations",
        [
            r"\bmultiple\b",
            r"\bone\s+pan\b",
            r"\bsame\s+pan\b",
            r"\bmore\s+than\s+one\b",
            r"\btwo\s+registrations\b",
        ],
    ),

    # Extra practical buckets often seen in Champions portal style data
    (
        "Delivery and Logistics",
        "Toolkit Delivery / Dispatch",
        [
            r"\btool\s*kit\b|\btoolkit\b",
            r"\bdeliver(y|ed)\b",
            r"\bdispatch(ed)?\b",
            r"\bnot\s*received\b",
        ],
    ),
    (
        "Complaint and Escalation",
        "Pending / Not Resolved",
        [
            r"\bpending\b",
            r"\bnot\s*resolved\b",
            r"\bno\s*response\b",
            r"\bescalat(e|ion)\b",
            r"\bcomplaint\b",
        ],
    ),
    (
        "Other Portal Services",
        "Status / Tracking",
        [
            r"\bstatus\b",
            r"\btrack\b|\btracking\b",
            r"\backnowledg(e|ment)\b",
        ],
    ),
    (
        "Other Portal Services",
        "Bank / Account Verification",
        [
            r"\bbank\b.*\bverification\b",
            r"\bifsc\b",
            r"\baccount\s*number\b",
        ],
    ),
    (
        "Other",
        "Unclear / Needs Manual Review",
        [r"^$"],  # empty text case
    ),
    # NOTE: any unmatched will fall to Other/Unclassified
 ]

def classify_grievance(desc: Optional[object], reply: Optional[object]) -> Tuple[str, str]:
    text = _norm(desc)
    # Some tickets are short in description but have signal in FinalReply; combine both.
    # Keep order: description is primary, reply adds context.
    text2 = _norm(reply)
    combined = (text + " " + text2).strip()
    for category, subcategory, patterns in RULES:
        for pat in patterns:
            if re.search(pat, combined):
                return category, subcategory
    return "Other", "Unclassified"

# Defensive checks
missing = [c for c in [DESC_COL, REPLY_COL] if c not in df.columns]
if missing:
    raise KeyError(f"Missing expected columns in df: {missing}. Available columns: {list(df.columns)}")

df[["GrievanceCategory", "GrievanceSubcategory"]] = df.apply(
    lambda r: pd.Series(classify_grievance(r[DESC_COL], r[REPLY_COL])), axis=1
)

# Quick sanity checks / analysis to support clustering workflow
display_cols = [c for c in [DESC_COL, REPLY_COL, "GrievanceCategory", "GrievanceSubcategory"] if c in df.columns]
df[display_cols].head(10)

# Category distribution (helps you see which groups are big)
df["GrievanceCategory"].value_counts(dropna=False).head(20)

# Category + Subcategory distribution
df.groupby(["GrievanceCategory", "GrievanceSubcategory"]).size().sort_values(ascending=False).head(30)

#df.loc[df["GrievanceCategory"].eq("Other"), display_cols].head(1000)
#df[df["GrievanceCategory"].eq("Technical and Portal Issues")][display_cols].head(1000)
df[df["GrievanceCategory"].eq("Eligibility and Documentation Queries")][display_cols].head(2000)
#Note: make these new derived columns in lowercase for uniformity

,GrievanceDescription,FinalReply,GrievanceCategory,GrievanceSubcategory
0,please find attachment letter head,sir please upload clear image aadhar card pan ...,Eligibility and Documentation Queries,Eligibility Criteria
1,udyam udyam br address wrong want cancel prere...,sir per request advised send details signed co...,Eligibility and Documentation Queries,Mandatory Documents
2,udyog aadhaar registration authority dear sir ...,grievance change registered mobile number requ...,Eligibility and Documentation Queries,Eligibility Criteria
3,regarding udyam registration even multiple att...,provided pan registered udyam udyam cg satan k...,Eligibility and Documentation Queries,Mandatory Documents
4,grievance regarding udyam registration even mu...,provided pan registered udyam udyam cg satan k...,Eligibility and Documentation Queries,Mandatory Documents
...,...,...,...,...
2188,res sir madam requested cancelled udyam certif...,sir cancelled register new urc correct details...,Eligibility and Documentation Queries,Mandatory Documents
2189,pan card error added udyam rj please cancel sa...,sir please contact nearest dfo relevant docume...,Eligibility and Documentation Queries,Mandatory Documents
2191,officer charge ministry micro small medium ent...,sir pan number editable portal please cancel e...,Eligibility and Documentation Queries,Eligibility Criteria
2192,udyog aadhar number registered mujhe yard nhi ...,sir udyam mp registration number linked mobile...,Eligibility and Documentation Queries,Mobile No./email Update
